In [1]:
import pandas as pd # type: ignore

look at the three datasets, using the Pandas read_csv function

In [4]:
area = pd.read_csv('state-areas.csv')
pop = pd.read_csv('state-population.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [13]:
# merge pop and abbrevs
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # drop duplicate information
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


Let’s double-check whether there were any mismatches here, which we can do by
looking for rows with nulls:

In [19]:
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

Some of the population values are null; let’s figure out which these are!

In [20]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


More importantly, we see that some of the new state entries are also null, which
means that there was no corresponding entry in the abbrevs key! Let’s figure out
which regions lack this match:

In [22]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico
(PR) and the United States as a whole (USA), while these entries do not appear in the
state abbreviation key. We can fix these quickly by filling in appropriate entries:

In [24]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool